In [22]:
# script to calculate ratios for use in TTS method 
# match rf 11 to varying BL for ratios 
# 1) RF11 UT to RF11 BL 
# 2) RF11 UT to campaign average BL 
# 3) RF11 UT to RF11, 10, 09 BL 
# author: sofia chelpon
# date created: 3-28-2020
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


In [2]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns
import datetime as dt
from scipy.optimize import least_squares

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [3]:
# load in both toga and awas data pickles 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df.pkl")

In [4]:
################# -------------------- TOGA  -------------------- #################

In [5]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
toga_means = pd.DataFrame()

# rf11 - UT 
toga_means['UT - RF11'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                                          & (toga_df['Flight'] == 'RF11')].mean()

# rf11 updraft region - BL 
toga_means['BL - RF11 Region'] = toga_df.loc[(toga_df['GGALT'] < 2000) 
                                      & (toga_df['GGLAT'] < 12) 
                                      & (toga_df['GGLON'] < 145)].mean()

# campaign average - BL 
toga_means['BL - All RF'] = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()


#toga_means.index.name = "Trace_Gas"
toga_means = toga_means.drop('GGALT').drop('GGLAT').drop('GGLON')
toga_means = toga_means.reset_index()
toga_means = toga_means.rename(columns={'index': 'Trace_Gas'})

In [6]:
region = toga_df.loc[(toga_df['GGALT'] < 2000) 
                                      & (toga_df['GGLAT'] < 12) 
                                      & (toga_df['GGLON'] < 145)]
region

,Time_UTC,Flight,Notes,Intrument,GGALT,GGLAT,GGLON,Acetaldehyde,n_Butane,i_Butane,Benzene,C3H8,Methanol,Bromoform,Tetrachloroethylene,Dibromomethane,Dichloromethane,Chloroform,Carbon_Tetrachloride
265,2014-01-19 03:05:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,1942.876831,8.146743,133.000229,268.000000,24.200001,17.850000,38.110001,193.0,975.0,0.918,1.70,0.921,74.309998,13.360000,80.599998
266,2014-01-19 03:07:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,1119.075195,8.301538,133.000061,270.799988,25.389999,19.320000,42.959999,160.0,914.0,1.045,1.88,0.984,82.360001,13.910000,81.500000
267,2014-01-19 03:09:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,303.940857,8.444719,133.000061,358.600006,25.309999,19.100000,52.849998,179.0,860.0,1.443,2.18,1.159,93.000000,15.890000,81.500000
268,2014-01-19 03:11:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,149.291580,8.582586,133.000198,315.299988,21.969999,18.070000,52.970001,190.0,792.0,1.358,1.86,1.292,88.250000,16.100000,84.699997
269,2014-01-19 03:13:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,148.558578,8.721257,133.000214,296.500000,20.430000,16.969999,54.770000,163.0,697.0,1.410,2.14,1.288,88.769997,16.370001,84.500000
270,2014-01-19 03:15:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,148.196671,8.860059,133.000061,279.299988,20.650000,16.889999,55.470001,165.0,695.0,1.638,2.14,1.202,87.220001,16.940001,84.500000
271,2014-01-19 03:17:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,148.673996,8.998617,133.000198,262.399994,20.020000,17.719999,55.610001,146.0,658.0,1.451,2.08,1.244,89.419998,16.420000,86.500000
272,2014-01-19 03:19:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,148.787323,9.136830,133.000214,256.500000,20.590000,15.240000,54.709999,164.0,681.0,1.391,2.29,1.165,86.139999,15.920000,88.300003
273,2014-01-19 03:21:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,764.482910,9.276450,133.000168,225.800003,17.610001,14.540000,50.439999,140.0,791.0,1.454,2.30,1.191,81.849998,14.900000,88.099998
274,2014-01-19 03:23:14,RF04,"Domain survey, 1 - Guam to SW (Palau)",TOGA,1742.250977,9.421556,133.000061,118.900002,8.120000,6.670000,25.100000,56.0,663.0,0.827,1.60,0.967,64.330002,12.010000,88.800003


In [28]:
# ratios for 1) flights in total and 2) individual flights 
toga_ratios = pd.DataFrame()

toga_ratios['Trace_Gas'] = toga_means['Trace_Gas']
toga_ratios['RF11_CampAvg'] = toga_means['UT - RF11']/toga_means['BL - All RF']
toga_ratios['RF11_Region'] = toga_means['UT - RF11']/toga_means['BL - RF11 Region']
toga_ratios

# drop species names so doesnt duplicate column when merged 
toga_ratios = toga_ratios.drop('Trace_Gas', axis = 1)

In [29]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_12162019.xlsx'
toga_lifetimes = pd.read_excel(mypath2)

# merge lifetimes to ratios
toga_ratios_full = toga_lifetimes.merge(toga_ratios, left_index=True, right_index=True)
toga_ratios_full.insert(0, 'Instrument', 'TOGA')

In [30]:
toga_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,RF11_CampAvg,RF11_Region
0,TOGA,Acetaldehyde,0.289063,0.365920,0.376967,0.330500,0.263911
1,TOGA,n_Butane,1.959376,3.813463,8.057685,0.121756,0.183413
2,TOGA,i_Butane,2.175570,4.031311,7.903592,0.193812,0.235234
3,TOGA,Benzene,3.755136,6.447323,11.947207,0.510844,0.390087
4,TOGA,C3H8,4.214557,9.062693,22.450000,0.369928,0.350480
5,TOGA,Methanol,5.160001,9.567463,20.040000,0.523531,0.446361
6,TOGA,Bromoform,15.069605,23.166062,33.432173,0.643168,0.601144
7,TOGA,Tetrachloroethylene,27.471720,77.860538,317.837040,0.704960,0.688547
8,TOGA,Dibromomethane,39.129546,95.533111,306.353687,0.941895,0.928691
9,TOGA,Dichloromethane,45.867749,109.800998,343.676023,0.870483,0.679986


In [31]:
################# -------------------- AWAS  -------------------- #################

In [32]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
awas_means = pd.DataFrame()

# rf11 - UT 
awas_means['UT - RF11'] = awas_df.loc[(toga_df['GGALT'] > 12000) & 
                                      (awas_df['GGALT'] < 14000) &
                                      (awas_df['Flight'] == 'RF11')].mean()

# rf11 updraft region - BL 
awas_means['BL - RF11 Region'] = awas_df.loc[(toga_df['GGALT'] < 2000) 
                                      & (awas_df['GGLAT'] < 12) 
                                      & (awas_df['GGLON'] < 145)].mean()

# campaign average - BL 
awas_means['BL - All RF'] = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()

#toga_means.index.name = "Trace_Gas"
awas_means = awas_means.drop('GGALT').drop('GGLAT').drop('GGLON')
awas_means = awas_means.reset_index()
awas_means = awas_means.rename(columns={'index': 'Trace_Gas'})

In [33]:
# ratios for 1) flights in total and 2) individual flights 
awas_ratios = pd.DataFrame()

awas_ratios['Trace_Gas'] = awas_means['Trace_Gas']
awas_ratios['RF11_CampAvg'] = awas_means['UT - RF11']/awas_means['BL - All RF']
awas_ratios['RF11_Region'] = awas_means['UT - RF11']/awas_means['BL - RF11 Region']

# drop species names so doesnt duplicate column when merged 
awas_ratios = awas_ratios.drop('Trace_Gas', axis = 1)

In [34]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_12162019.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
len(awas_lifetimes)

# merge lifetimes to ratios 
awas_ratios_full = awas_lifetimes.merge(awas_ratios, left_index=True, right_index=True)
awas_ratios_full.insert(0, 'Instrument', 'AWAS')

In [35]:
awas_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,RF11_CampAvg,RF11_Region
0,AWAS,C2H6_Ethane,18.555721,48.949092,167.260709,0.638492,1.034435
1,AWAS,C2H2_Ethyne,5.422335,9.603369,18.712675,0.626241,1.076144
2,AWAS,C3H8_Propane,4.214557,9.062693,22.450000,0.217952,0.939052
3,AWAS,C4H10_Isobutane,2.175570,4.031311,7.903592,0.120980,0.944104
4,AWAS,C4H10_nButane,1.959376,3.813463,8.057685,0.111386,0.789474
5,AWAS,C5H12_nPentane,1.214449,2.312606,4.723079,0.074720,0.972222
6,AWAS,C6H6_Benzene,3.755136,6.447323,11.947207,0.675202,1.553560
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,0.984468,0.993492
8,AWAS,C2H6S_Dimethyl_sulfide,0.955122,1.689426,3.279163,0.221208,10.363247
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,1.007016,1.003556


In [36]:
################# -------------------- MERGE THE TWO  -------------------- #################

In [39]:
master_list = awas_ratios_full
master_list = master_list.append(toga_ratios_full, ignore_index = True, sort = False)

# pickle this!! 
master_list.to_pickle("./contrast_ratios_rf11_regional.pkl")

# look at output 
#master_list[master_list['Trace_Gas'] == 'Dichloromethane'].drop('BL_tau', axis = 1).drop('UT_tau', axis = 1)

In [40]:
master_list

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,RF11_CampAvg,RF11_Region
0,AWAS,C2H6_Ethane,18.555721,48.949092,167.260709,0.638492,1.034435
1,AWAS,C2H2_Ethyne,5.422335,9.603369,18.712675,0.626241,1.076144
2,AWAS,C3H8_Propane,4.214557,9.062693,22.450000,0.217952,0.939052
3,AWAS,C4H10_Isobutane,2.175570,4.031311,7.903592,0.120980,0.944104
4,AWAS,C4H10_nButane,1.959376,3.813463,8.057685,0.111386,0.789474
5,AWAS,C5H12_nPentane,1.214449,2.312606,4.723079,0.074720,0.972222
6,AWAS,C6H6_Benzene,3.755136,6.447323,11.947207,0.675202,1.553560
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,0.984468,0.993492
8,AWAS,C2H6S_Dimethyl_sulfide,0.955122,1.689426,3.279163,0.221208,10.363247
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,1.007016,1.003556
